In [1]:
import pandas as pd
import os

import requests
import json
import numpy as np
from time import sleep
import re
from copy import deepcopy

# improt zip-file stuff
import unicodedata
from zipfile import ZipFile
from bs4 import BeautifulSoup as bsp
from io import StringIO, BytesIO
import json

In [2]:
working_directory = os.getcwd()

# Download FDA_DB.zip

In [5]:
os.mkdir(os.path.join(working_directory, '0_zip_files'))
os.mkdir(os.path.join(working_directory, '1_unzipped_current_files'))

In [7]:
url = 'https://www.fda.gov/drugs/drug-approvals-and-databases/drugsfda-data-files'
url_req = requests.get(url, headers={'User-Agent':'Custom'})
new_str = unicodedata.normalize("NFKD", url_req.text)
soup = bsp(new_str, 'html.parser')

link = soup.find('div', class_='col-md-8 col-md-push-2')

link_fin = 'https://www.fda.gov' + link.ul.li.a['href']
sleep(3)
r = requests.get(link_fin, stream=True)

with open(os.path.join(working_directory, '0_zip_files', f'FDA_database.zip'), 'wb') as f:
    f.write(r.content)

with ZipFile(os.path.join(working_directory, '0_zip_files', f'FDA_database.zip'), 'r') as zipObj:
    zipObj.extractall(os.path.join(working_directory, '1_unzipped_current_files'))

# Prepare extracted files

In [8]:
df_mark_st = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/MarketingStatus.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)
df_product = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/Products.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)
df_types = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/Applications.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)


C:\Users\user79\AppData\Local\Temp\ipykernel_4724\688377870.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_mark_st = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/MarketingStatus.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)
C:\Users\user79\AppData\Local\Temp\ipykernel_4724\688377870.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_product = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/Products.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)
Skipping line 35350: expected 8 fields, saw 9
Skipping line 35351: expected 8 fields, saw 9
Skipping line 35352: expected 8 fields, saw 9
Skipping line 43879: expected 8 fields, saw 9
Skipping line 43981: expected 8 fields, saw 9
Skipping line 43982: expected 8 fields, saw

In [9]:
df_product

,ApplNo,ProductNo,Form,Strength,ReferenceDrug,DrugName,ActiveIngredient,ReferenceStandard
0,4,4,SOLUTION/DROPS;OPHTHALMIC,1%,0,PAREDRINE,HYDROXYAMPHETAMINE HYDROBROMIDE,0.0
1,159,1,TABLET;ORAL,500MG,0,SULFAPYRIDINE,SULFAPYRIDINE,0.0
2,552,1,INJECTABLE;INJECTION,"20,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
3,552,2,INJECTABLE;INJECTION,"40,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
4,552,3,INJECTABLE;INJECTION,"5,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
...,...,...,...,...,...,...,...,...
45246,761289,1,INJECTABLE;INTRAVENOUS,25MG/1.25ML(20MG/ML),0,IMJUDO,TREMELIMUMAB-ACTL,0.0
45247,761289,2,INJECTABLE;INTRAVENOUS,300MG/15ML(20MG/ML),0,IMJUDO,TREMELIMUMAB-ACTL,0.0
45248,761291,1,INJECTABLE;INJECTION,30MG/3ML(10MG/ML),0,TECVAYLI,TECLISTAMAB-CQYV,0.0
45249,761291,2,INJECTABLE;INJECTION,153MG/1.7ML(90MG/ML),0,TECVAYLI,TECLISTAMAB-CQYV,0.0


In [11]:
df_mark_st['MarketingStatusID'] = df_mark_st['MarketingStatusID'].replace(3, 'Discontinued').replace(1, 'Prescription').replace(2, 'Over-the-counter').replace(4, 'None (Tentative Approval)')

In [57]:
df_from_FDA = df_product.merge(df_mark_st, how='left').merge(df_types, how='left').drop_duplicates()

In [13]:
df_from_FDA

,ApplNo,ProductNo,Form,Strength,ReferenceDrug,DrugName,ActiveIngredient,ReferenceStandard,MarketingStatusID,ApplType,ApplPublicNotes,SponsorName
0,4,4,SOLUTION/DROPS;OPHTHALMIC,1%,0,PAREDRINE,HYDROXYAMPHETAMINE HYDROBROMIDE,0.0,Discontinued,NDA,NaN,PHARMICS
1,159,1,TABLET;ORAL,500MG,0,SULFAPYRIDINE,SULFAPYRIDINE,0.0,Discontinued,NDA,NaN,LILLY
2,552,1,INJECTABLE;INJECTION,"20,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0,Discontinued,NDA,NaN,ORGANON USA INC
3,552,2,INJECTABLE;INJECTION,"40,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0,Discontinued,NDA,NaN,ORGANON USA INC
4,552,3,INJECTABLE;INJECTION,"5,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0,Discontinued,NDA,NaN,ORGANON USA INC
...,...,...,...,...,...,...,...,...,...,...,...,...
45246,761289,1,INJECTABLE;INTRAVENOUS,25MG/1.25ML(20MG/ML),0,IMJUDO,TREMELIMUMAB-ACTL,0.0,Prescription,BLA,NaN,ASTRAZENECA AB
45247,761289,2,INJECTABLE;INTRAVENOUS,300MG/15ML(20MG/ML),0,IMJUDO,TREMELIMUMAB-ACTL,0.0,Prescription,BLA,NaN,ASTRAZENECA AB
45248,761291,1,INJECTABLE;INJECTION,30MG/3ML(10MG/ML),0,TECVAYLI,TECLISTAMAB-CQYV,0.0,Prescription,BLA,NaN,JANSSEN BIOTECH
45249,761291,2,INJECTABLE;INJECTION,153MG/1.7ML(90MG/ML),0,TECVAYLI,TECLISTAMAB-CQYV,0.0,Prescription,BLA,NaN,JANSSEN BIOTECH


In [14]:
# applDocs
df_appl_docs = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/ApplicationDocs.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)
df_appl_docs_just_labels = df_appl_docs[df_appl_docs['ApplicationDocsTypeID']==2]

C:\Users\user79\AppData\Local\Temp\ipykernel_4724\347543185.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_appl_docs = pd.read_csv(os.path.join(working_directory, '1_unzipped_current_files/ApplicationDocs.txt'), sep='\t', encoding = "cp1252", error_bad_lines=False)


In [16]:
df_appl_docs_just_labels['ApplicationDocsDate'] = pd.to_datetime(df_appl_docs_just_labels['ApplicationDocsDate'])

C:\Users\user79\AppData\Local\Temp\ipykernel_4724\2789387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appl_docs_just_labels['ApplicationDocsDate'] = pd.to_datetime(df_appl_docs_just_labels['ApplicationDocsDate'])


In [56]:
from tqdm import tqdm
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore")
    pd.options.mode.chained_assignment = None

    df_appl_docs_latest_marked = pd.DataFrame()
    for applNo, df in tqdm(df_appl_docs_just_labels.groupby('ApplNo')):
        df_to_add = deepcopy(df)
        df_sort = df_to_add.sort_values('ApplicationDocsDate', ascending=False).reset_index(drop=True)
        # df_sort.loc[0, 'LatestFDALabel'] = True
        df_appl_docs_latest_marked = df_appl_docs_latest_marked.append(df_sort.loc[[0]])

100%|█████████████████████████████████████████████████████████████████████████████| 4547/4547 [00:10<00:00, 443.51it/s]


In [59]:
df_appl_docs_latest_marked_fin = df_appl_docs_latest_marked[['ApplNo', 'ApplicationDocsURL']]

In [60]:
df_from_FDA = df_from_FDA.merge(df_appl_docs_latest_marked_fin, how='left')

In [40]:
# df_from_FDA['LatestFDALabel'] = df_from_FDA['LatestFDALabel'].fillna(False)

In [74]:
df_from_FDA_fin = df_from_FDA[
                            (df_from_FDA['MarketingStatusID']!='Discontinued')&
                            # (df_from_FDA['ApplType'].isin(['NDA', 'BLA']))&
                            (df_from_FDA['ApplicationDocsURL'].notnull())
                            ].drop(['ProductNo', 'Strength', 'ReferenceDrug', 'ReferenceStandard', 
                                    'MarketingStatusID', 'ApplPublicNotes'], axis = 1).drop_duplicates(subset=['ApplNo'])
df_from_FDA_fin

,ApplNo,Form,DrugName,ActiveIngredient,ApplType,SponsorName,ApplicationDocsURL
78,4782,TABLET;ORAL,PREMARIN,"ESTROGENS, CONJUGATED",NDA,WYETH PHARMS,http://www.accessdata.fda.gov/drugsatfda_docs/...
92,5213,TABLET;ORAL,HYCODAN,HOMATROPINE METHYLBROMIDE; HYDROCODONE BITARTRATE,NDA,GENUS,http://www.accessdata.fda.gov/drugsatfda_docs/...
106,5378,TABLET;ORAL,DESOXYN,METHAMPHETAMINE HYDROCHLORIDE,NDA,KEY THERAP,http://www.accessdata.fda.gov/drugsatfda_docs/...
140,5939,INJECTABLE;INJECTION,BAL,DIMERCAPROL,NDA,AKORN,http://www.accessdata.fda.gov/drugsatfda_docs/...
151,6035,INJECTABLE;INJECTION,METHERGINE,METHYLERGONOVINE MALEATE,NDA,EDISON THERAPS LLC,http://www.accessdata.fda.gov/drugsatfda_docs/...
...,...,...,...,...,...,...,...
45242,761269,INJECTABLE;INJECTION,LEQEMBI,LECANEMAB-IRMB,BLA,EISAI INC,https://www.accessdata.fda.gov/drugsatfda_docs...
45244,761270,"INJECTABLE;INTRAVENOUS, SUBCUTANEOUS",IMJUDO,TREMELIMUMAB-ACTL,BLA,ASTRAZENECA AB,https://www.accessdata.fda.gov/drugsatfda_docs...
45246,761289,INJECTABLE;INTRAVENOUS,IMJUDO,TREMELIMUMAB-ACTL,BLA,ASTRAZENECA AB,http://www.accessdata.fda.gov/drugsatfda_docs/...
45248,761291,INJECTABLE;INJECTION,TECVAYLI,TECLISTAMAB-CQYV,BLA,JANSSEN BIOTECH,http://www.accessdata.fda.gov/drugsatfda_docs/...


In [75]:
df_from_FDA_fin[df_from_FDA_fin['ActiveIngredient'].str.contains('TRASTUZUMAB')]

,ApplNo,Form,DrugName,ActiveIngredient,ApplType,SponsorName,ApplicationDocsURL
29384,103792,VIAL; INTRAVENOUS,HERCEPTIN,TRASTUZUMAB,BLA,GENENTECH,http://www.accessdata.fda.gov/drugsatfda_docs/...
29482,125427,VIAL; SINGLE-USE,KADCYLA,ADO-TRASTUZUMAB EMTANSINE,BLA,GENENTECH,http://www.accessdata.fda.gov/drugsatfda_docs/...
45101,761073,VIAL; MULTIDOSE,KANJINTI,TRASTUZUMAB-ANNS,BLA,AMGEN INC,http://www.accessdata.fda.gov/drugsatfda_docs/...
45103,761074,INJECTABLE;INJECTION,OGIVRI,TRASTUZUMAB-DKST,BLA,MYLAN GMBH,http://www.accessdata.fda.gov/drugsatfda_docs/...
45114,761081,INJECTABLE;INJECTION,TRAZIMERA,TRASTUZUMAB-QYYP,BLA,PFIZER INC,http://www.accessdata.fda.gov/drugsatfda_docs/...
45131,761091,INJECTABLE;INJECTION,HERZUMA,TRASTUZUMAB-PKRB,BLA,CELLTRION INC,http://www.accessdata.fda.gov/drugsatfda_docs/...
45137,761100,POWDER;IV (INFUSION),ONTRUZANT,TRASTUZUMAB-DTTB,BLA,SAMSUNG BIOEPIS CO LTD,http://www.accessdata.fda.gov/drugsatfda_docs/...
45145,761106,INJECTABLE;SUBCUTANEOUS,HERCEPTIN HYLECTA,TRASTUZUMAB; HYALURONIDASE-OYSK,BLA,GENENTECH INC,http://www.accessdata.fda.gov/drugsatfda_docs/...
45173,761139,INJECTABLE;INJECTION,ENHERTU,FAM-TRASTUZUMAB DERUXTECAN-NXKI,BLA,DAIICHI SANKYO,http://www.accessdata.fda.gov/drugsatfda_docs/...
45192,761170,INJECTABLE;SUBCUTANEOUS,PHESGO,PERTUZUMAB;TRASTUZUMAB;HYALURONIDASE-ZZXF,BLA,GENENTECH INC,https://www.accessdata.fda.gov/drugsatfda_docs...


In [77]:
df_from_FDA_fin.to_csv(os.path.join(working_directory, 'df_fda_short.csv'), index=False)
df_from_FDA.to_csv(os.path.join(working_directory, 'df_fda_full.csv'), index=False)

In [3]:
# df_from_FDA_fin = pd.read_csv(os.path.join(working_directory, 'df_fda_short.csv'))

In [4]:
df_from_FDA_fin.columns = ['ApplNo', 'Form', 'Drug', 'ActiveIngredient', 'ApplType',
                           'Sponsor', 'LatestLabel']

In [5]:
df_from_FDA_fin['LinkToFDA'] = df_from_FDA_fin['ApplNo'].apply(lambda x: 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=' + str(x) )
df_from_FDA_fin['ApplNo'] = df_from_FDA_fin['ApplNo'].apply(lambda x: str(x))

In [8]:
drug_fda_dict = {drugName:df.to_dict('records') for drugName, df in df_from_FDA_fin.groupby('ActiveIngredient')}

In [9]:
drug_fda_dict.update({drugName:df.to_dict('records') for drugName, df in df_from_FDA_fin.groupby('Drug')})

In [14]:
drug_fda_dict.get('TRASTUZUMAB')

[{'ApplNo': '103792',
  'Form': 'VIAL; INTRAVENOUS',
  'Drug': 'HERCEPTIN',
  'ActiveIngredient': 'TRASTUZUMAB',
  'ApplType': 'BLA',
  'Sponsor': 'GENENTECH',
  'LatestLabel': 'http://www.accessdata.fda.gov/drugsatfda_docs/label/2018/103792s5345lbl.pdf',
  'LinkToFDA': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=103792'}]

In [10]:
applno_fda_dict = {drugName:df.to_dict('records')[0] for drugName, df in df_from_FDA_fin.groupby('ApplNo')}

In [12]:
with open(os.path.join(working_directory, 'drug_fda_dict.json'), 'w') as f:
    json.dump(drug_fda_dict, f)
    
with open(os.path.join(working_directory, 'applno_fda_dict.json'), 'w') as f:
    json.dump(applno_fda_dict, f)